In [1]:
import requests
import os

To obtain a reddit client id and secret key, visit https://www.reddit.com/prefs/apps
Reddit secrets prefixed with: `REDDIT_`
- `DEV_NAME` can either be your reddit username or the username of your bot
- `BOT_USER` username of bot
- `BOT_PASS` password of bot
- `CLIENT_ID` application client (listed under application name)
- `SECRET_KEY` application api key

In [18]:
# os.environ["OPENAI_API_KEY"] = ""
# os.environ["REDDIT_DEV_NAME"] = "" 
# os.environ["REDDIT_BOT_USER"] = "" 
# os.environ["REDDIT_BOT_PASS"] = "" 
# os.environ["REDDIT_CLIENT_ID"] = "" 
# os.environ["REDDIT_SECRET_KEY"] = "" 


In [3]:
open_api_key = os.environ.get('OPENAI_API_KEY')
reddit_dev = os.environ.get('REDDIT_DEV_NAME')

#### Setup Reddit Auth Headers

In [ ]:
agent = f"shell:com.example.myredditapp:v0.0.1 (by /u/{reddit_dev})"
auth_headers = {'User-Agent': agent}

#### Basic user authentication

In [ ]:
client_id = os.environ.get('REDDIT_CLIENT_ID')
client_secret = os.environ.get('REDDIT_SECRET_KEY')
username = os.environ.get('REDDIT_BOT_USER')
password = os.environ.get('REDDIT_BOT_PASS')

# First, authenticate and get an access token
auth_url = 'https://www.reddit.com/api/v1/access_token'
auth_data = {'grant_type': 'password', 'username': username, 'password': password}
auth_response = requests.post(auth_url, headers=auth_headers, data=auth_data, auth=(client_id, client_secret))
access_token = auth_response.json()['access_token']


In [ ]:
# Now that we have an access token, let's make a request to get information about the authenticated user
user_url = 'https://oauth.reddit.com/api/v1/me'
user_headers = {'Authorization': f'Bearer {access_token}', 'User-Agent': agent}
user_response = requests.get(user_url, headers=user_headers)

print(user_response.json())

#### Create our RedditSearch Tool 

In [10]:
from langchain.agents import tool
from langchain.tools import BaseTool

class RedditWrapper(BaseTool):
    name = "RedditSearch"
    description = "Useful for finding subreddits from the Reddit API"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return self.search_reddit_for(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

    @tool("search", return_direct=True)
    def search_reddit_for(search_query: str, max_results: int=10):
        """Searches the Reddit API for the query."""
        url = f'https://www.reddit.com/subreddits/search.json?q={search_query}&limit={max_results}'

        response = requests.get(url, headers=auth_headers)
        res = []

        if response.status_code == 200:
            data = response.json()['data']
            for subreddit in data['children']:
                result_row = (subreddit['data']['display_name']+"\t"+
                    subreddit['data']['name'] + "\t" + 
                    subreddit['data']['url'])
                print(result_row)
                res.append(result_row.replace("\t", " "))        
        else:
            print(f'Request failed with status code {response.status_code}')
        
        return res

### Quick Test for redditSearch

In [11]:
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import OpenAI
redditSearch = RedditWrapper()

llm = OpenAI(temperature=0.9)
search_term = "News"
urls = redditSearch.search_reddit_for(search_term)
prompt = PromptTemplate(input_variables=["urls", "query"],
                                   template="Review the following list of Redit Pages. Formatted as title id path.\n\
                                             {urls}\n\
                                             What is the most relevant path for a search for: {query}.\n")

prompt.format(query=search_term, urls=urls)


news	t5_2qh3l	/r/news/
worldnews	t5_2qh13	/r/worldnews/
newsbabes_nsfw	t5_2e3cg0	/r/newsbabes_nsfw/
politics	t5_2cneq	/r/politics/
Conservative	t5_2qh6p	/r/Conservative/
NewsPorn	t5_2ski5	/r/NewsPorn/
NewsOfTheWeird	t5_2vmjd	/r/NewsOfTheWeird/
conspiracy	t5_2qh4r	/r/conspiracy/
NewsAroundYou	t5_6arjr2	/r/NewsAroundYou/
NewsHub	t5_3ifah	/r/NewsHub/


"Review the following list of Redit Pages. Formatted as title id path.\n                                             ['news t5_2qh3l /r/news/', 'worldnews t5_2qh13 /r/worldnews/', 'newsbabes_nsfw t5_2e3cg0 /r/newsbabes_nsfw/', 'politics t5_2cneq /r/politics/', 'Conservative t5_2qh6p /r/Conservative/', 'NewsPorn t5_2ski5 /r/NewsPorn/', 'NewsOfTheWeird t5_2vmjd /r/NewsOfTheWeird/', 'conspiracy t5_2qh4r /r/conspiracy/', 'NewsAroundYou t5_6arjr2 /r/NewsAroundYou/', 'NewsHub t5_3ifah /r/NewsHub/']\n                                             What is the most relevant path for a search for: News.\n"

### Use redditSearch with an agent

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import OpenAI
redditSearch = RedditWrapper()

llm = OpenAI(temperature=0)

tools = [
    Tool(
        name="RedditSearch",
        func=redditSearch.run, 
        description="A Reddit Search Engine. Useful when you want to find subbreddits'",
    )
]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
topic = "News"

prompt = PromptTemplate(input_variables=["query"],
                                  template="What is the most relevant path for a search for: {query}.\n")

agent.run(prompt.format(query=topic))




> Entering new AgentExecutor chain...
 I should look for a subreddit that is dedicated to news.
Action: RedditSearch
Action Input: Newsnews	t5_2qh3l	/r/news/
UpliftingNews	t5_2u3ta	/r/UpliftingNews/
NewsPorn	t5_2ski5	/r/NewsPorn/
politics	t5_2cneq	/r/politics/
worldnews	t5_2qh13	/r/worldnews/
NewsOfTheWeird	t5_2vmjd	/r/NewsOfTheWeird/
NewsOfTheStupid	t5_2tpr6	/r/NewsOfTheStupid/
conspiracy	t5_2qh4r	/r/conspiracy/
Conservative	t5_2qh6p	/r/Conservative/
NewsAroundYou	t5_6arjr2	/r/NewsAroundYou/

Observation: ['news t5_2qh3l /r/news/', 'UpliftingNews t5_2u3ta /r/UpliftingNews/', 'NewsPorn t5_2ski5 /r/NewsPorn/', 'politics t5_2cneq /r/politics/', 'worldnews t5_2qh13 /r/worldnews/', 'NewsOfTheWeird t5_2vmjd /r/NewsOfTheWeird/', 'NewsOfTheStupid t5_2tpr6 /r/NewsOfTheStupid/', 'conspiracy t5_2qh4r /r/conspiracy/', 'Conservative t5_2qh6p /r/Conservative/', 'NewsAroundYou t5_6arjr2 /r/NewsAroundYou/']
Thought: I should look for a subreddit that is dedicated to news and is highly active.
Actio

'The most relevant path for a search for news is /r/worldnews/.'